In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np

Define some constants

In [ ]:
run = "veka10"
path = Path("..", "..", "resources", run)
fn_ratios = "industry_sector_ratios_[YEAR].csv"
fn_mapping = "20240320_industry_ratios_mapping.csv"
fn_output = "20240320_industry_ratios_output_[COUNTRY].xlsx"
years = [2020, 2030, 2040, 2050]
country = "BE"

Read sector ratios data from `resources` for each year. Add mapping with defined industry.

In [ ]:
dfx = []
for y in years:
    df = (
        pd.read_csv(Path(path, fn_ratios.replace("[YEAR]", str(y))), header=[0,1], index_col=0)
        [country]
        .T
    )
    df["year"] = y
    df.index.name = "technology"
    dfx.append(df)
mapping = pd.read_csv(fn_mapping, header=0, index_col=0)
dfx = (
    pd.concat(dfx)
    .join(mapping, how="left")
    .reset_index()
    .set_index(["industry", "technology", "year"])
    .sort_values(by=["industry", "technology", "year"])
)

Define a display function to output data

In [ ]:
def display_technology(df, industry):
    df = (
        df.query("industry == @industry")
        .replace(0, np.nan)
        .dropna(axis=1, how="all")
        .loc[industry]
    )
    df.index.rename({"technology": "MWh / t"}, inplace=True)
    return df

In [ ]:
display_technology(dfx, "Iron and Steel")

Write output to file

In [ ]:
with pd.ExcelWriter(fn_output.replace("[COUNTRY]", country)) as writer:
    for industry in dfx.index.get_level_values(0).drop_duplicates():
        display_technology(dfx, industry).to_excel(writer, sheet_name=industry)